Кобзарь О.С. Хабибуллин Р.А. 2019 г.

# Модель фонтанирующей скважины

In [ ]:
import sys
sys.path.append('../')

import uniflocpy.uWell.deviation_survey as dev_sev
import uniflocpy.uTools.data_workflow as utool
import uniflocpy.uMultiphaseFlow.gas_flow as gas_flow
import uniflocpy.uTools.uconst as uconst
import uniflocpy.uWell.uPipe as Pipe
import uniflocpy.uWell.Self_flow_well as self_flow_well
import plotly.graph_objs as go
import numpy as np
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import tools
import pandas as pd
init_notebook_mode(connected=True)
import scipy.interpolate as interpolate
import matplotlib.pyplot as plt
data = utool.Data()
from uniflocpy.uTools import plotly_workflow
import re
import uniflocpy.uPVT.BlackOil_model as BlackOil_model
import uniflocpy.uTemperature as uTemperature
import uniflocpy.uReservoir.IPR_simple_line as IPR_simple_line
import uniflocpy.uTools.plotly_workflow as plotly_workflow 
import uniflocpy.uValidation.python_api as python_api
import uniflocpy.uValidation.by_UniflocVBA as bvba
from shapely.geometry import LineString


## Настройки расчета

In [ ]:
calc_options ={"step_lenth_in_calc_along_wellbore_m":25,
                "without_annulus_space":False,
                "solver_using":True}


def calc_energy_kjm3(p_tube_bar, p_wellhead_bar, r_gas_injected_m3m3):
    result = (1000 * (1.2/0.2) * r_gas_injected_m3m3 * 0.1 * 
              (
                (p_tube_bar/p_wellhead_bar)**(0.24) - 1    
                )
             )
    return result

def find_intersection(result_df, column1_name, column2_name):
    x = result_df[column1_name].dropna().index.values
    f = result_df[column1_name].dropna().values

    x2 = result_df[column2_name].dropna().index.values
    g = result_df[column2_name].dropna().values
    first_line = LineString(np.column_stack((x, f)))
    second_line = LineString(np.column_stack((x2, g)))
    intersection = first_line.intersection(second_line)
    try:
        return [intersection.xy[0][0], intersection.xy[1][0]]
    except:
        return [0, 0]

In [ ]:
calc_energy_kjm3(84.4029695013113, 11.1, 75)

## Данные по скважинам

Скв. 2Ф Исследование 5РМ

In [ ]:
1.45  * 24.05 / 28.98

In [ ]:
1.45 /1.293

Скважина исходная

In [ ]:

well_name = '2Ф. Ромашкинское м.'
pb_bar = 90
fluid_data = {"rsb_m3m3" : 56,
"gamma_oil" : 0.86,
 "gamma_gas" : 1.45  * 24.05 / 28.98,
#"gamma_gas" : 1.45 /1.293,  #добавить содержание метана и азота

'pb_bar': pb_bar}

well_data = {"h_intake_mes_m" :1205.5,
"h_intake_vert_m" : 1205.5,
"h_bottomhole_mes_m" : 1605, 
"h_bottomhole_vert_m": 1605,
             
"geothermal_grad_cm":0.02,
"t_wellhead_c":20,
"t_bottomhole_c" :40,
"t_earth_init_in_reservoir_c" : 40,
'p_bottomhole_bar': 114.35, 
             'p_wellhead_bar':9,
"d_casing_inner_m" : 0.133,
"d_tube_inner_m" : 0.0503,
"qliq_on_surface_m3day":40,
"p_reservoir_bar":177,
"fw_on_surface_perc":0}
real_measurements = pd.DataFrame({'p_survey_mpa': [0.9, 1.12, 1.83, 2.957, 4.355, 5.785, 7.3, 8.953, 9.863, 10.176, 11.435],
                                 'h_mes_survey_m': [0, 105, 305, 505, 705, 905, 1105, 1305, 1405, 1505, 1605]})

Скважина обводненная c газлифтом

In [ ]:
well_name = '2Ф. Ромашкинское м.'
pb_bar = 90
fluid_data = {"rsb_m3m3" : 56,
"gamma_oil" : 0.86,
 "gamma_gas" : 1.45  * 24.05 / 28.98,
#"gamma_gas" : 1.45 /1.293,

'pb_bar': pb_bar}

well_data_glw = {"h_intake_mes_m" :1205.5,
"h_intake_vert_m" : 1205.5,
"h_bottomhole_mes_m" : 1605, 
"h_bottomhole_vert_m": 1605,
             
"geothermal_grad_cm":0.02,
                 
"t_wellhead_c":20,
"t_bottomhole_c" :40,
"t_earth_init_in_reservoir_c" : 40,
'p_bottomhole_bar': 114.35, 
             'p_wellhead_bar':9,
"d_casing_inner_m" : 0.0503,
"d_tube_inner_m" : 0.0503,
"qliq_on_surface_m3day":40,
"p_reservoir_bar":177,
"fw_on_surface_perc":60,
"multiplier_for_pi":0.8}
real_measurements = pd.DataFrame({'p_survey_mpa': [0.9, 1.12, 1.83, 2.957, 4.355, 5.785, 7.3, 8.953, 9.863, 10.176, 11.435],
                                 'h_mes_survey_m': [0, 105, 305, 505, 705, 905, 1105, 1305, 1405, 1505, 1605]})

Исследование 18ТМ Скв.2Ф

In [ ]:
well_name = '2Ф. Туймазинское м.'
pb_bar = 90
fluid_data = {"rsb_m3m3" : 54.5,
"gamma_oil" : 0.852,
#"gamma_gas" : 1.45  * 24.05 / 28.98,
               "gamma_gas" : 1.45  * 24.05 / 28.98,

'pb_bar': pb_bar}

well_data = {"h_intake_mes_m" :1205,
"h_intake_vert_m" : 1205,
"h_bottomhole_mes_m" : 1650, 
"h_bottomhole_vert_m": 1650,
             
"geothermal_grad_cm":0.027,
"t_wellhead_c":23.2,
"t_bottomhole_c" :40,
"t_earth_init_in_reservoir_c" : 40,
'p_bottomhole_bar':120,
 'p_wellhead_bar':11.1,
"d_casing_inner_m" : 0.1553,
"d_tube_inner_m" : 0.062,
"qliq_on_surface_m3day":176.5,
"p_reservoir_bar":180,
"fw_on_surface_perc":0}
real_measurements = pd.DataFrame({'p_survey_mpa': [1.11, 1.86, 2.97, 4.17, 5.59, 7.02, 8.55, 10.10, 12.0],
                                 'h_mes_survey_m': [0, 200, 400, 600, 800, 1000, 1200, 1400, 1650]})

Газлифтная скважина

In [ ]:
well_name = '2Ф. Туймазинское м.'
pb_bar = 90
fluid_data = {"rsb_m3m3" : 54.5,
"gamma_oil" : 0.852,
#"gamma_gas" : 1.45  * 24.05 / 28.98,
               "gamma_gas" : 1.45  * 24.05 / 28.98,

'pb_bar': pb_bar}

well_data_glw = {"h_intake_mes_m" :1205,
"h_intake_vert_m" : 1205,
"h_bottomhole_mes_m" : 1650, 
"h_bottomhole_vert_m": 1650,
             
"geothermal_grad_cm":0.027,
"t_wellhead_c":23.2,
"t_bottomhole_c" :40,
"t_earth_init_in_reservoir_c" : 40,
'p_bottomhole_bar':120,
 'p_wellhead_bar':11.1,
"d_casing_inner_m" : 0.062,
"d_tube_inner_m" : 0.062,
"qliq_on_surface_m3day":176.5,
"p_reservoir_bar":180,
"fw_on_surface_perc":50,
    "multiplier_for_pi":0.8}
real_measurements = pd.DataFrame({'p_survey_mpa': [1.11, 1.86, 2.97, 4.17, 5.59, 7.02, 8.55, 10.10, 12.0],
                                 'h_mes_survey_m': [0, 200, 400, 600, 800, 1000, 1200, 1400, 1650]})

## Настройка модели скважины и расчет

Модель флюида Мищенко

In [ ]:
def create_well(fluid_data, well_data, calc_options, true_rsb_m3t=False, pains_corr_using=0,
               gravity_grad_coef=1,friction_grad_coef=1, epsilon_friction_m=0.00029):
    blackoil_option = BlackOil_model.BlackOil_option()
    blackoil_option.set_vba_preset()
    simple_well = self_flow_well.self_flow_well(fluid=1, reservoir = 0, pipe=0, temp_corr=1, **fluid_data,
                                                    **well_data, **calc_options, activate_rus_mode = 1)
    simple_well.pipe.fluid_flow.fl.option = blackoil_option
    
    simple_well.pipe.fluid_flow.fl.true_rsb_m3t = true_rsb_m3t
    simple_well.pipe.hydr_cor.pains_corr_using =pains_corr_using
    simple_well.pipe.hydr_cor.gravity_grad_coef =gravity_grad_coef
    simple_well.pipe.hydr_cor.friction_grad_coef =friction_grad_coef
    simple_well.pipe.hydr_cor.epsilon_friction_m = epsilon_friction_m
    return simple_well

In [ ]:
true_rsb_m3t = False
pains_corr_using =1
gravity_grad_coef =0.925
friction_grad_coef =0.9
epsilon_friction_m = 0.00029

In [ ]:
#simple_well.pipe.fluid_flow.fl.gas_dissolved_m3t_fixed = 70.66        #72.2 false                  #67.22 true
#simple_well.pipe.fluid_flow.fl.rs_m3m3_fixed = 60.2                 #62.2                        #57.81  


In [ ]:
simple_well = create_well(fluid_data, well_data,
                          calc_options, 
                          true_rsb_m3t=true_rsb_m3t,
                          pains_corr_using=pains_corr_using,
               gravity_grad_coef=gravity_grad_coef,
                          friction_grad_coef=friction_grad_coef, 
                          epsilon_friction_m=epsilon_friction_m)

In [ ]:
simple_well = create_well(fluid_data, well_data_glw,
                          calc_options, 
                          true_rsb_m3t=true_rsb_m3t,
                          pains_corr_using=pains_corr_using,
               gravity_grad_coef=gravity_grad_coef,
                          friction_grad_coef=friction_grad_coef, 
                          epsilon_friction_m=epsilon_friction_m)

In [ ]:
simple_well.pipe.fluid_flow.fl.gas_dissolved_m3t_fixed = 71.76       #72.2 false                  #67.22 true
simple_well.pipe.fluid_flow.fl.rs_m3m3_fixed = 61.13                #62.2                        #57.81  


In [ ]:
simple_well.calc_all_from_down_to_up() #снизу-вверх

In [ ]:
simple_well.calc_all_from_up_to_down() #сверху-вниз

In [ ]:
new_pi_m3daybar = simple_well.ipr.pi_m3daybar * simple_well.multiplier_for_pi
print(simple_well.ipr.pi_m3daybar, simple_well.multiplier_for_pi, new_pi_m3daybar)

## Выгрузка расчитанных значений и объединение с фактическими данными

In [ ]:
result_df = simple_well.data.get_data_as_df()
result_df.to_excel('uvalidation_self_flow_well_result.xlsx')
result_df = result_df.set_index('well.h_calculated_mes_m')

In [ ]:
result_df_p = simple_well.data.get_data_as_df()
result_df_p = result_df_p.set_index('well.p_calculated_bar')

Выгрузка ограниченного набора данных

In [ ]:
group_patterns = [['p_calculated_bar'], ['t_calculated'],['gas_fraction'], ['rhon'], ['mun'], ['density'], 
                  ['friction_grad'], ['number_re'], ['pb_bar'], ['fluid_flow', 'msec'], ['vsl'], ['vsg'], ['result_grad'], 
                  ['acceleration'], ['rsb_m3t'], ['gas_liberated_m3t'], ['gas_dissolved_m3t'],
                 ['b_oil_m3m3'], ['rho_oil_kgm3'], ['mu_oil_cp'],['fl.z'], ['rho_gas_dissolved_relative_d']]
antipatterns = ['temp_cor', 'dvdp', 'grad_coef', 'module_friction', 'grad_using', 'for_rus_cor']
print_all = True
columns_to_plot = plotly_workflow.create_columns_to_plot(result_df, group_patterns, antipatterns, print_all)
short_df = result_df[columns_to_plot]
short_df = utool.rename_columns_by_dict(short_df)

class_names = ['hydr_cor.', 'temp_cor.', 'fl.', 'fluid_flow.', 'well.', 'earth']
for i in class_names:
    short_df = utool.rename_columns_by_dict(short_df,{'':i})
short_df.index.name = 'Измеренная глубина, м'
short_df.to_excel('uvalidation_self_flow_well_short_result.xlsx')

In [ ]:
result_df_one_well = result_df.copy()

Добавление результатов фактических исследований

In [ ]:
real_measurements["p_survey_bar"] = real_measurements["p_survey_mpa"] * 10
real_measurements = real_measurements.set_index(real_measurements['h_mes_survey_m'])
real_measurements.index.name = 'well.h_calculated_mes_m'

result_df = result_df.join(real_measurements, how = 'outer')

In [ ]:
(real_measurements["p_survey_bar"][0] - simple_well.p_wellhead_bar) / real_measurements["p_survey_bar"][0] * 100  #снизу-вверх

In [ ]:
(real_measurements["p_survey_bar"][simple_well.h_bottomhole_mes_m] - 
 simple_well.p_bottomhole_bar) / real_measurements["p_survey_bar"][simple_well.h_bottomhole_mes_m] * 100 #сверху-вниз

## Построение графиков

In [ ]:
group_patterns = [["p_bar", 'fl'], ['t_c', 'fl'], ['well.t_calculated_earth_init'], ['survey'], ['p_calculated_bar_vba'], ['t_calculated_c_vba']]
antipatterns = ['mu', 'h_mes_survey', 'mpa', 'fl.', 'earth']
plotly_workflow.plot_by_patterns(result_df, group_patterns, antipatterns, plot_name='Распределение свойств по глубине',
                                 print_all=False)

In [ ]:
plotly_workflow.plot_by_patterns(result_df_p,  [['rho_oil_kgm3']], ['cor_number', 'cal'], swap_xy=False, reversed_y=False, 
                                 plot_name='Плотность газонасыщенной нефти',
                                 x_name = 'Давление, бар', 
                                y_name = 'Плотность, кг/м3',
                                traces_names = [
                                               'Плотность, кг/м3',
                                               ]
                                )

In [ ]:
group_patterns = [["gas_dissolved_m3t"],['gas_liberated_m3t']]
antipatterns = ['tpb', 'number']
plotly_workflow.plot_by_patterns(result_df_p, group_patterns, antipatterns, plot_name='Зависимость параметров от давления',
                                 print_all=False, swap_xy=False, reversed_y=False)

In [ ]:
group_patterns = [["liberated_gas_sc_m3m3"],['dissolved_gas_sc_m3m3']]
antipatterns = ['tpb', 'number']
plotly_workflow.plot_by_patterns(result_df_p, group_patterns, antipatterns,
                                 print_all=False, swap_xy=False, reversed_y=False,
                                plot_name='Удельный объем выделившегося и растворенного газа',
                                x_name = 'Давление, бар', 
                                y_name = 'м3/м3',
                                traces_names = [
                                               'Удельный объем выделившегося газа, м3/м3',
                                    'Удельный объем растворенного газа, м3/м3'
                                               ])

In [ ]:
group_patterns = [["p_calculated_bar"], ['p_survey']]
antipatterns = ['mu', 'h_mes_survey', 'mpa','fl','ipr']
plotly_workflow.plot_by_patterns(result_df, group_patterns, antipatterns, plot_name='КРД',
                                 print_all=False,
                                x_name='Давление, бар',
    y_name='Глубина, м',
    traces_names=['Расчет КРД по Беггз Брилл', 'Факт КРД'])

In [ ]:
plotly_workflow.plot_by_patterns(result_df_p,  [['b_oil_m3m3']], ['cor_number', 'cal'], swap_xy=False, reversed_y=False, 
                                 plot_name='Объемный коэффициент газонасыщенной нефти',
                                 x_name = 'Давление, бар', 
                                y_name = 'bн, м3/м3',
                                traces_names = [
                                               'bн, м3/м3',
                                               ]
                                )

In [ ]:
plotly_workflow.plot_by_patterns(result_df_p,  [['mu_oil_cp']], ['cor_number', 'cal'], 
                                 swap_xy=False, reversed_y=False,  plot_name='Вязкость газонасыщенной нефти',
                                x_name = 'Давление, бар', 
                                y_name = 'Вязкость, сПуаз',
                                traces_names = [
                                               'Вязкость нефти, сПуаз',
                                               ])

In [ ]:
result_df_phi = result_df_p.copy()
result_df_phi = result_df_phi.set_index('fluid_flow.gas_fraction_d')
group_patterns = [["gas_fraction_real_d"]]
antipatterns = ['tpb', 'number']
plotly_workflow.plot_by_patterns(result_df_phi, group_patterns, antipatterns,
                                 print_all=False, swap_xy=False, reversed_y=False,
                                plot_name='Истинная доля газа от расходной',
                                x_name = 'Расходная доля газа, д.ед. (Без учета проскальзывания св. газа)', 
                                y_name = 'Истинная доля газа, д.ед. (с учетом проскальзывания св. газа)',
                                traces_names = [
                                               'Вязкость нефти, сПуаз',
                                               ])

In [ ]:
plotly_workflow.plot_by_patterns(result_df_p,  [['rho_gas_liberated']], ['cor_number', 'cal'], swap_xy=False, reversed_y=False, 
                                 plot_name='Относительная плотность выделившегося газа',
                                 x_name = 'Давление, бар', 
                                y_name = 'Относительная плотность',
                                traces_names = [
                                               'Вязкость, сПуаз',
                                               ]
                                )

## Расчет сверху-вниз для разных Rг (удельного количество закачиваемого газа

In [ ]:
simple_well_for_variant = simple_well = create_well(fluid_data, well_data_glw,
                          calc_options, 
                          true_rsb_m3t=true_rsb_m3t,
                          pains_corr_using=pains_corr_using,
               gravity_grad_coef=gravity_grad_coef,
                          friction_grad_coef=friction_grad_coef, 
                          epsilon_friction_m=epsilon_friction_m)

In [ ]:
result_df_to_plot = None
results = []
r_injected_list = list(range(0, 176, 25))
for i in r_injected_list:
    print(i)
    simple_well_for_variant.data.clear_data()
    simple_well_for_variant.pipe.fluid_flow.r_gas_injected_m3m3 = i
    simple_well_for_variant.calc_all_from_up_to_down()
    this_result_df = simple_well_for_variant.data.get_data_as_df()
    this_result_df = this_result_df.set_index('well.h_calculated_mes_m')
    results.append(this_result_df)
    
    this_result_df = this_result_df.add_prefix(f"Rg: {i} м3/м3, ")
    try:
        result_df_to_plot = result_df_to_plot.join(this_result_df)
    except:
        result_df_to_plot = this_result_df.copy()

Результаты в одном DataFrame

In [ ]:
result_df_one_well = result_df_one_well[result_df_one_well['well.p_calculated_bar']>1.1]
result_df = result_df_one_well.join(result_df_to_plot, how='outer')

Расчет распределения давленя в затрубе

In [ ]:
result_p_annulus_bar = []
list_of_mes  = list(range(0, well_data_glw["h_bottomhole_mes_m"]+(calc_options["step_lenth_in_calc_along_wellbore_m"]-1),
               calc_options["step_lenth_in_calc_along_wellbore_m"]))
list_of_mes = result_df.index.values
for i in list_of_mes:
    h_mes_m = i
    p_annulus_bar = gas_flow.calc_p_in_annulus_bar(h_m=i,
                                                   p_wellhead_bar=60, 
                                                   t_wellhead_c=20, 
                                                   gamma_gas=0.8,
                                                   t_bottomhole_c=40,
                                                   h_bottomhole_m=well_data_glw["h_bottomhole_mes_m"])
    result_p_annulus_bar.append(p_annulus_bar)
    
annulus_df = pd.DataFrame({'p_calculated_bar_annulus':result_p_annulus_bar},
                         list_of_mes)

def fix_p_annulus(annulus_df = annulus_df):
    old_values = annulus_df['p_calculated_bar_annulus'].values
    new_values = []
    for i, j in enumerate(old_values):
        if i>1 and i < 30:
            new_values.append(None)
        else:
            new_values.append(j)
    annulus_df['p_calculated_bar_annulus'] = new_values
    annulus_df = annulus_df.interpolate('spline',order = 1, axis=0, limit_direction ='both')
    return annulus_df
annulus_df = fix_p_annulus(annulus_df)
annulus_df.plot()

In [ ]:
result_df = result_df.join(annulus_df, how='outer')

## Продолжение построения распределения свойств по глубине для выбранной скважины

In [ ]:
result_df_to_super_plot = result_df.copy()
this_watercut_str = well_data_glw['fw_on_surface_perc']
group_patterns = [["p_calculated_bar"]]
antipatterns = ['mu', 'h_mes_survey', 'mpa','fl','ipr']
dict_to_translate = {f"м3/м3, КРД сверху-вниз В={this_watercut_str}%":'well.p_calculated_bar',
                    #f"КРД снизу-вверх В={this_watercut_str}%":'well.p_calculated_bar',
                    f" ":'well.p_calculated_bar',
                    'Rг:': 'Rg:',
                    "КРД в затрубе":"p_calculated_bar_annulus"}
columns_to_plot = plotly_workflow.create_columns_to_plot(result_df_to_super_plot, group_patterns, antipatterns, False)
print(columns_to_plot)
result_traces_names = columns_to_plot
for z,i in enumerate(result_traces_names):
    for j in dict_to_translate.items():
        if j[1] in i:
            new_name = i.replace(j[1], j[0])
            result_traces_names[z] =new_name
result_traces_names[0] = f"КРД снизу-вверх В={this_watercut_str}%"
for i, j in enumerate(result_traces_names):
    result_traces_names[i] = j.replace(', well.p_calculated_bar','')
print(result_traces_names)


In [ ]:
group_patterns = [["p_calculated_bar"]]
antipatterns = ['mu', 'h_mes_survey', 'mpa','fl','ipr']
plotly_workflow.plot_by_patterns(result_df_to_super_plot, group_patterns, antipatterns,
                                 plot_name=(f"Расчет КРД для скважины {well_name}: В = {this_watercut_str}%, " +
                                           f"Кпрод.(B={this_watercut_str})/Кпрод.(B=0)={well_data_glw['multiplier_for_pi']}"),
                                 print_all=True,
                                x_name='Давление, бар',
    y_name='Глубина, м',                         
    traces_names=result_traces_names
                                        )
plotly_workflow.plot_by_patterns(result_df_to_super_plot, group_patterns, antipatterns,
                                 plot_name=(f"Расчет КРД для скважины {well_name.replace('.','')}, В = {this_watercut_str}%"),
                                 print_all=True,
                                x_name='Давление, бар',
                                 iplot_option = False,
    y_name='Глубина, м',                         
    traces_names=result_traces_names 
                                        )

In [ ]:
overall_result = None
for i in r_injected_list:
    h_mes_m, p_tube_bar = find_intersection(result_df, 'well.p_calculated_bar', f"Rg: {i} м3/м3, well.p_calculated_bar")
    
    x = result_df['p_calculated_bar_annulus'].dropna().index.values
    y = result_df['p_calculated_bar_annulus'].dropna().values
    f = interpolate.interp1d(x, y)
    p_annulus_bar = f(h_mes_m)
    
    if p_annulus_bar>p_tube_bar>0:
        check = 'ИСТИНА'
        energy = calc_energy_kjm3(p_tube_bar, well_data_glw['p_wellhead_bar'],i)
    else:
        check = "ЛОЖЬ"
        energy = 0
    
    
    
    this_result = pd.DataFrame({'Расход газа, м3м3':i,
                                'Глубина точки ввода газа, м':h_mes_m,
                               'Давление в НКТ, бар':p_tube_bar,
                               'Давление в затрубе, бар': p_annulus_bar,
                               'Давление в затрубе больше':check,
                               'УРЭ, кДж/м3':energy},
                              index = [i])
    try:
        overall_result =  overall_result.append(this_result)
    except:
        overall_result = this_result.copy()
if overall_result[overall_result['УРЭ, кДж/м3']>0].shape[0]>0:        
    optimum_r_gas_injected_m3m3 = overall_result[overall_result['УРЭ, кДж/м3']>0]['УРЭ, кДж/м3'].idxmin()
else:
    optimum_r_gas_injected_m3m3 = -1
result_column = []
for i in overall_result.index.values:
    if i == optimum_r_gas_injected_m3m3:
        result = 'Оптимальный режим'
    else:
        result = 'Неоптимальный режим'
    result_column.append(result)

overall_result['Итог'] = result_column
overall_result.to_excel('uWell.GLW_injection_point.xlsx')
overall_result

In [ ]:
overall_result['УРЭ, кДж/м3'].plot()

In [ ]:
h_injection_point_m = overall_result[overall_result.index==optimum_r_gas_injected_m3m3]['Глубина точки ввода газа, м'][optimum_r_gas_injected_m3m3]
p_injection_point_bar = overall_result[overall_result.index==optimum_r_gas_injected_m3m3]['Давление в НКТ, бар'][optimum_r_gas_injected_m3m3]

bottom_piece_df = result_df_one_well[result_df_one_well.index > h_injection_point_m]
bottom_piece_df = bottom_piece_df.sort_index()

top_piece_df_columns = plotly_workflow.create_columns_to_plot(result_df_to_plot, [f" {optimum_r_gas_injected_m3m3}"],[])
top_piece_df = result_df_to_plot[top_piece_df_columns]
class_names = [f"Rg: {optimum_r_gas_injected_m3m3} м3/м3, "]
for i in class_names:
    top_piece_df = utool.rename_columns_by_dict(top_piece_df,{'':i})
top_piece_df = top_piece_df.sort_index()


top_piece_df = top_piece_df[top_piece_df.index <=h_injection_point_m]
result_df_glw = top_piece_df.append(bottom_piece_df)


group_patterns = [['r_gas_injected_m3m3'],['p_calculated_bar'], ['t_calculated'],['gas_fraction'], ['rhon'], ['mun'], ['density'], 
                  ['friction_grad'], ['number_re'], ['pb_bar'], ['fluid_flow', 'msec'], ['vsl'], ['vsg'], ['result_grad'], 
                  ['acceleration'], ['rsb_m3t'], ['gas_liberated_m3t'], ['gas_dissolved_m3t'],
                 ['b_oil_m3m3'], ['rho_oil_kgm3'], ['mu_oil_cp'],['fl.z'], ['rho_gas_dissolved_relative_d']]
antipatterns = ['temp_cor', 'dvdp', 'grad_coef', 'module_friction', 'grad_using', 'for_rus_cor','true', 'fixed']
print_all = True
columns_to_plot = plotly_workflow.create_columns_to_plot(result_df_glw, group_patterns, antipatterns, print_all)
short_df = result_df_glw[columns_to_plot]
short_df = utool.rename_columns_by_dict(short_df)

class_names = ['hydr_cor.', 'temp_cor.', 'fl.', 'fluid_flow.', 'well.', 'earth', f"Rg: {optimum_r_gas_injected_m3m3} м3/м3, "]
for i in class_names:
    short_df = utool.rename_columns_by_dict(short_df,{'':i})
short_df.index.name = 'Измеренная глубина, м'
short_df.to_excel('GLW.parameters.xlsx')

In [ ]:
last_plot = result_df_one_well.copy()
top_piece_df_columns = plotly_workflow.create_columns_to_plot(result_df_to_plot, [f" {optimum_r_gas_injected_m3m3}"],[])
top_piece_df = result_df_to_plot[top_piece_df_columns]
last_plot = last_plot.join(top_piece_df, how = 'outer')
last_plot = last_plot.join(annulus_df, how = 'outer')

group_patterns = [["p_calculated_bar"]]
antipatterns = ['mu', 'h_mes_survey', 'mpa','ipr']
plotly_workflow.plot_by_patterns(last_plot, group_patterns, antipatterns, 
                                 plot_name=(f"Оптимальный режим работы для {well_name}: В = {this_watercut_str}%, " +
                                           f"Кпрод.(B={this_watercut_str}%)/Кпрод.(B=0%)={well_data_glw['multiplier_for_pi']}"),
                                 print_all=True,
                                x_name='Давление, бар',
    y_name='Глубина, м',
    traces_names=[f"КРД снизу-вверх В={this_watercut_str}%",
                  f'Rг: {optimum_r_gas_injected_m3m3} м3/м3',
                  'КРД в затрубе']
                                )
p_injection_point_bar_str = str(p_injection_point_bar)
p_injection_point_bar_str_splitted = p_injection_point_bar_str.split('.')
p_injection_point_bar_str = p_injection_point_bar_str_splitted[0] + '.' +  p_injection_point_bar_str_splitted[0][:2]

p_injection_point_bar_str = str(round(p_injection_point_bar, 2))

h_injection_point_m_str = str(h_injection_point_m)
h_injection_point_m_str_splitted = h_injection_point_m_str.split('.')
h_injection_point_m_str = h_injection_point_m_str_splitted[0] + '.' +  h_injection_point_m_str_splitted[0][:2]

h_injection_point_m_str = str(round(h_injection_point_m, 2))


plotly_workflow.plot_by_patterns(last_plot, group_patterns, antipatterns, 
                                 plot_name=(f"Оптимальный режим работы для скважины {well_name} В = {this_watercut_str}%"),
                                 print_all=True,
                                x_name='Давление, бар',
    y_name='Глубина, м',
    traces_names=[f"КРД снизу-вверх В={this_watercut_str}%",
                  f'Rг: {optimum_r_gas_injected_m3m3} м3/м3',
                  'КРД в затрубе'],
                                 iplot_option = False,
                        annotation = {'x': p_injection_point_bar , 'y':h_injection_point_m,
                                      'text': f"Ввод газа. H = {h_injection_point_m_str} м, P = {p_injection_point_bar_str} бар" }
                                )